# import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.1.0


# Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & pre-processing function

In [3]:

def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

# Performance function

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)    
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

# Model function

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

# Create model(class)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class,self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()
        
        self.model = tf.keras.Sequential()
        self.model.add(flatten())
        
        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())
            
        self.model.add(dense(label_dmi, weight_init))
        
    def call(self, x, training=None, maks=None):
        x = self.model(x)
        return x

# Create model(function)

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.RandomNormal()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

# Define data & hyper parameter

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

# Define model & optimizer & writer

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_relu'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

# Restore checkpoint & start train or test phase

In [10]:

if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.8869, train_loss: 2.15588474, train_accuracy: 0.4766, test_Accuracy: 0.2907
Epoch: [ 0] [    1/  468] time: 0.9699, train_loss: 2.12282038, train_accuracy: 0.5312, test_Accuracy: 0.4461
Epoch: [ 0] [    2/  468] time: 1.0529, train_loss: 2.03110743, train_accuracy: 0.6328, test_Accuracy: 0.5613
Epoch: [ 0] [    3/  468] time: 1.1279, train_loss: 1.95565116, train_accuracy: 0.6250, test_Accuracy: 0.5958
Epoch: [ 0] [    4/  468] time: 1.2059, train_loss: 1.89365554, train_accuracy: 0.6797, test_Accuracy: 0.5973
Epoch: [ 0] [    5/  468] time: 1.2799, train_loss: 1.86772275, train_accuracy: 0.5781, test_Accuracy: 0.6019
Epoch: [ 0] [    6/  468] time: 1.3579, train_loss: 1.75229454, train_accuracy: 0.5859, test_Accuracy: 0.6199
Epoch: [ 0] [    7/  468] time: 1.4349, train_loss: 1.62631893, train_accuracy: 0.6406, test_Accuracy: 0.6361
Epoch: [ 0] [    8/  468] time: 1.5099,

Epoch: [ 0] [   75/  468] time: 6.7619, train_loss: 0.32459348, train_accuracy: 0.8750, test_Accuracy: 0.9055
Epoch: [ 0] [   76/  468] time: 6.8459, train_loss: 0.31547764, train_accuracy: 0.9062, test_Accuracy: 0.9089
Epoch: [ 0] [   77/  468] time: 6.9219, train_loss: 0.31134102, train_accuracy: 0.8906, test_Accuracy: 0.9115
Epoch: [ 0] [   78/  468] time: 6.9969, train_loss: 0.31994036, train_accuracy: 0.8984, test_Accuracy: 0.9102
Epoch: [ 0] [   79/  468] time: 7.0819, train_loss: 0.22662675, train_accuracy: 0.9062, test_Accuracy: 0.9094
Epoch: [ 0] [   80/  468] time: 7.1599, train_loss: 0.25000298, train_accuracy: 0.9219, test_Accuracy: 0.9091
Epoch: [ 0] [   81/  468] time: 7.2359, train_loss: 0.31339902, train_accuracy: 0.9062, test_Accuracy: 0.9108
Epoch: [ 0] [   82/  468] time: 7.3109, train_loss: 0.25301409, train_accuracy: 0.9297, test_Accuracy: 0.9098
Epoch: [ 0] [   83/  468] time: 7.3869, train_loss: 0.48464227, train_accuracy: 0.8906, test_Accuracy: 0.9099
Epoch: [ 0

Epoch: [ 0] [  150/  468] time: 12.7723, train_loss: 0.28057763, train_accuracy: 0.9219, test_Accuracy: 0.9307
Epoch: [ 0] [  151/  468] time: 12.8553, train_loss: 0.26176965, train_accuracy: 0.9297, test_Accuracy: 0.9304
Epoch: [ 0] [  152/  468] time: 12.9333, train_loss: 0.23602587, train_accuracy: 0.9297, test_Accuracy: 0.9307
Epoch: [ 0] [  153/  468] time: 13.0123, train_loss: 0.24601766, train_accuracy: 0.9141, test_Accuracy: 0.9302
Epoch: [ 0] [  154/  468] time: 13.0943, train_loss: 0.24171469, train_accuracy: 0.9375, test_Accuracy: 0.9301
Epoch: [ 0] [  155/  468] time: 13.1713, train_loss: 0.13322899, train_accuracy: 0.9766, test_Accuracy: 0.9315
Epoch: [ 0] [  156/  468] time: 13.2503, train_loss: 0.18741947, train_accuracy: 0.9453, test_Accuracy: 0.9311
Epoch: [ 0] [  157/  468] time: 13.3353, train_loss: 0.21392885, train_accuracy: 0.9141, test_Accuracy: 0.9323
Epoch: [ 0] [  158/  468] time: 13.4133, train_loss: 0.20662133, train_accuracy: 0.9531, test_Accuracy: 0.9329
E

Epoch: [ 0] [  225/  468] time: 18.7883, train_loss: 0.16200928, train_accuracy: 0.9375, test_Accuracy: 0.9411
Epoch: [ 0] [  226/  468] time: 18.8713, train_loss: 0.21792501, train_accuracy: 0.9375, test_Accuracy: 0.9416
Epoch: [ 0] [  227/  468] time: 18.9493, train_loss: 0.13543305, train_accuracy: 0.9531, test_Accuracy: 0.9418
Epoch: [ 0] [  228/  468] time: 19.0253, train_loss: 0.21468611, train_accuracy: 0.9375, test_Accuracy: 0.9414
Epoch: [ 0] [  229/  468] time: 19.1093, train_loss: 0.18278299, train_accuracy: 0.9453, test_Accuracy: 0.9425
Epoch: [ 0] [  230/  468] time: 19.1873, train_loss: 0.16407694, train_accuracy: 0.9688, test_Accuracy: 0.9427
Epoch: [ 0] [  231/  468] time: 19.2635, train_loss: 0.19368020, train_accuracy: 0.9609, test_Accuracy: 0.9411
Epoch: [ 0] [  232/  468] time: 19.3485, train_loss: 0.19154121, train_accuracy: 0.9453, test_Accuracy: 0.9408
Epoch: [ 0] [  233/  468] time: 19.4275, train_loss: 0.25482425, train_accuracy: 0.9453, test_Accuracy: 0.9411
E

Epoch: [ 0] [  300/  468] time: 24.8019, train_loss: 0.14845161, train_accuracy: 0.9609, test_Accuracy: 0.9428
Epoch: [ 0] [  301/  468] time: 24.8829, train_loss: 0.11147936, train_accuracy: 0.9688, test_Accuracy: 0.9471
Epoch: [ 0] [  302/  468] time: 24.9619, train_loss: 0.08035105, train_accuracy: 0.9844, test_Accuracy: 0.9494
Epoch: [ 0] [  303/  468] time: 25.0449, train_loss: 0.30172193, train_accuracy: 0.9141, test_Accuracy: 0.9470
Epoch: [ 0] [  304/  468] time: 25.1319, train_loss: 0.15328255, train_accuracy: 0.9453, test_Accuracy: 0.9473
Epoch: [ 0] [  305/  468] time: 25.2169, train_loss: 0.21160980, train_accuracy: 0.9375, test_Accuracy: 0.9477
Epoch: [ 0] [  306/  468] time: 25.2989, train_loss: 0.13312802, train_accuracy: 0.9609, test_Accuracy: 0.9482
Epoch: [ 0] [  307/  468] time: 25.3889, train_loss: 0.18372877, train_accuracy: 0.9453, test_Accuracy: 0.9497
Epoch: [ 0] [  308/  468] time: 25.4639, train_loss: 0.15371415, train_accuracy: 0.9609, test_Accuracy: 0.9508
E

Epoch: [ 0] [  375/  468] time: 30.7769, train_loss: 0.19331932, train_accuracy: 0.9531, test_Accuracy: 0.9525
Epoch: [ 0] [  376/  468] time: 30.8529, train_loss: 0.16559793, train_accuracy: 0.9375, test_Accuracy: 0.9515
Epoch: [ 0] [  377/  468] time: 30.9309, train_loss: 0.17650959, train_accuracy: 0.9609, test_Accuracy: 0.9512
Epoch: [ 0] [  378/  468] time: 31.0129, train_loss: 0.19316274, train_accuracy: 0.9453, test_Accuracy: 0.9508
Epoch: [ 0] [  379/  468] time: 31.0899, train_loss: 0.16883585, train_accuracy: 0.9688, test_Accuracy: 0.9489
Epoch: [ 0] [  380/  468] time: 31.1649, train_loss: 0.23728526, train_accuracy: 0.9141, test_Accuracy: 0.9484
Epoch: [ 0] [  381/  468] time: 31.2529, train_loss: 0.12373729, train_accuracy: 0.9688, test_Accuracy: 0.9481
Epoch: [ 0] [  382/  468] time: 31.3369, train_loss: 0.15088041, train_accuracy: 0.9375, test_Accuracy: 0.9488
Epoch: [ 0] [  383/  468] time: 31.4129, train_loss: 0.12194327, train_accuracy: 0.9609, test_Accuracy: 0.9494
E

Epoch: [ 0] [  450/  468] time: 36.7009, train_loss: 0.19894055, train_accuracy: 0.9375, test_Accuracy: 0.9631
Epoch: [ 0] [  451/  468] time: 36.7769, train_loss: 0.08746324, train_accuracy: 0.9688, test_Accuracy: 0.9631
Epoch: [ 0] [  452/  468] time: 36.8559, train_loss: 0.08986873, train_accuracy: 0.9766, test_Accuracy: 0.9632
Epoch: [ 0] [  453/  468] time: 36.9339, train_loss: 0.09490823, train_accuracy: 0.9609, test_Accuracy: 0.9636
Epoch: [ 0] [  454/  468] time: 37.0109, train_loss: 0.07166877, train_accuracy: 0.9766, test_Accuracy: 0.9637
Epoch: [ 0] [  455/  468] time: 37.0869, train_loss: 0.15674700, train_accuracy: 0.9531, test_Accuracy: 0.9638
Epoch: [ 0] [  456/  468] time: 37.1679, train_loss: 0.07939424, train_accuracy: 0.9766, test_Accuracy: 0.9628
Epoch: [ 0] [  457/  468] time: 37.2449, train_loss: 0.07525711, train_accuracy: 0.9844, test_Accuracy: 0.9622
Epoch: [ 0] [  458/  468] time: 37.3229, train_loss: 0.14014688, train_accuracy: 0.9453, test_Accuracy: 0.9616
E

# Test Accuracy : 95.94%